In [1]:
!pip install PyPDF2 sentence_transformers chromadb langchain langchain-groq

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        api_key="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",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

In [3]:
import PyPDF2
import os

text_by_page=[]
def extract_text_from_pdf_with_pages(pdf_file):
    
    try:
        with open(pdf_file, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page_number, page in enumerate(reader.pages):
                text_by_page.append((page_number + 1, page.extract_text(),pdf_file))  # Page numbers are 1-indexed
    except Exception as e:
        print(f"Error reading {pdf_file}: {e}")
    return text_by_page


In [4]:
pdf_file="genai-principles.pdf"
text_by_page = extract_text_from_pdf_with_pages(pdf_file)

In [5]:
print(text_by_page)

[(1, ' Karan Singh, Assistant Professor of Operations Research \nPrinciples of Generative AI A Technical Introduction Generative artificial intelligence (GenAI) tools are an emerging class of new-age artificial intelligence algorithms capable of producing novel content — in varied formats such as text, audio, video, pictures, and code — based on user prompts. Recent advances in machine learning (ML), massive datasets, and substantial increases in computing power have propelled such tools to human-level performance on academic and professional benchmarks, 1comparable to the ninetieth percentile on the SAT and the bar exam. This rapid progress has led many to to believe that the metamorphosis of these technologies 2from research-grade demos to accessible and easy-to-use production-grade goods and services carries the potential to supercharge business processes and operations while enabling entirely new deliverables heretofore rendered infeasible by economic or technological factors. It t

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
CHUNK_SIZE=10000
OVERLAP_SIZE=1000
def chunk_text_with_pages(text_by_page, chunk_size=CHUNK_SIZE, overlap_size=OVERLAP_SIZE):
    chunks_with_metadata = []
    splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n","\n"],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = chunk_size,  # size of each chunk created
    chunk_overlap  = overlap_size,  # size of  overlap between chunks in order to maintain the context
    length_function = len  # Function to calculate size, currently we are using "len" which denotes length of string however you can pass any token counter)
    )
    for page_number, text,pdf_file, in text_by_page:
        chunks = splitter.split_text(text)
        for chunk in chunks:
            chunks_with_metadata.append({"chunk": chunk, "page_number": page_number, "file_name": pdf_file})
    return chunks_with_metadata


In [7]:
# Chunk text using LangChain
chunks_with_metadata = chunk_text_with_pages(text_by_page, CHUNK_SIZE, OVERLAP_SIZE)

In [ ]:
chunks_with_metadata